In [2]:
import  SimpleITK as sitk
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
# from downloaddata import fetch_data, fetch_data_all

image_viewer = sitk.ImageViewer()
image_viewer.SetApplication('C:/Program Files/ITK-SNAP 4.2/bin/ITK-SNAP.exe')

image=sitk.ReadImage("C:/Users/User/Desktop/Task/3702_left_knee.nii.gz")
#image_viewer.Execute(image)


In [58]:
image_array=sitk.GetArrayFromImage(image)

In [59]:
# body_mask = sitk.BinaryThreshold(image, lowerThreshold=-150, upperThreshold=1500, insideValue=1, outsideValue=0)
# cc = sitk.ConnectedComponent(body_mask)
# stats = sitk.LabelShapeStatisticsImageFilter()
# stats.Execute(cc)

# largest_label = max(stats.GetLabels(), key=lambda l: stats.GetPhysicalSize(l))
# largest_component_mask = sitk.BinaryThreshold(cc, lowerThreshold=largest_label, upperThreshold=largest_label, insideValue=1, outsideValue=0)
# large=sitk.BinaryClosingByReconstruction(largest_component_mask,[5,5,5],sitk.sitkBall)


# body=sitk.GetArrayFromImage(large)

In [60]:
# cleaned_image=image_array*body
# clean=sitk.GetImageFromArray(cleaned_image)
# clean.SetOrigin(image.GetOrigin())
# clean.SetSpacing(image.GetSpacing())

In [61]:
# sitk.WriteImage(cleaned_image_masking,'clean_mask.nii.gz')

In [62]:
# plt.imshow(body[200,:,:],cmap='gray')

In [3]:
main_image=sitk.ReadImage('clean.nii.gz')

In [4]:
main=sitk.GetArrayFromImage(main_image)
np.max(main)

1769

In [5]:
feature_img = sitk.GradientMagnitude(main_image)


In [ ]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=20, markWatershedLine=False, fullyConnected=False)


In [172]:
sitk.WriteImage(ws_img,'ws_imag.nii.gz')

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [9]:
import nibabel as nib
import numpy as np
import cv2
import os

# Load NIfTI file
nifti_path = 'clean.nii.gz'
nifti_img = nib.load(nifti_path)
volume = nifti_img.get_fdata()
affine = nifti_img.affine

# Output: list of marker coordinates in image space
marker_coords = []

# Iterate through axial slices (or other orientations if needed)
for z in range(volume.shape[2]):
    slice_img = volume[:, :, z]
    
    # Normalize to 0-255 for OpenCV
    norm_slice = cv2.normalize(slice_img, None, 0, 255, cv2.NORM_MINMAX)
    norm_slice = norm_slice.astype(np.uint8)

    # Threshold to highlight bright regions (adjust as needed)
    _, binary = cv2.threshold(norm_slice, 200, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        # Approximate roundness via area and perimeter
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * (area / (perimeter * perimeter))

        # Keep only circular-looking shapes
        if 0.7 < circularity < 1.2 and area > 20:  # adjust area threshold
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                marker_coords.append((cx, cy, z))

print(f"Found {len(marker_coords)} ring markers.")

# Optionally: save markers as a binary volume (same shape as CT)
marker_volume = np.zeros_like(volume, dtype=np.uint8)
for x, y, z in marker_coords:
    marker_volume[x, y, z] = 1  # or a label value

# Save marker volume to NIfTI
marker_img = nib.Nifti1Image(marker_volume, affine)
nib.save(marker_img, 'ring_markers.nii.gz')


ModuleNotFoundError: No module named 'cv2'